In [1]:
import os

os.chdir(r"/home/yousinator/AuspicesAI/ScytheEx/ai")

from internal_lib import DataSetInformation, MachineLearning
import pandas as pd

In [2]:
titles = [
    "dur",
    "proto",
    "service",
    "state",
    "stcpb",
    "dtcpb",
    "synack",
    "ackdat",
    "trans_depth",
    "response_body_len",
    "ct_src_dport_ltm",
    "ct_dst_sport_ltm",
    "is_ftp_login",
    "ct_ftp_cmd",
    "ct_flw_http_mthd",
    "is_sm_ips_ports",
    "attack_cat",
]


train = pd.read_csv("data/training.csv")
test = pd.read_csv("data/testing.csv")

combined_df = pd.concat([train, test], ignore_index=True)

# Shuffle the combined DataFrame
df = combined_df.sample(frac=1).reset_index(drop=True)

df = df[titles]

df_info = DataSetInformation(df)

df.drop_duplicates(inplace=True)

In [15]:
services = df["service"].unique()



array(['dns', '-', 'http', 'ssh', 'smtp', 'ftp-data', 'ftp', 'pop3',
       'ssl', 'dhcp', 'snmp', 'irc', 'radius'], dtype=object)

In [19]:
df["state"].unique()

array(['CON', 'INT', 'ECO', 'FIN', 'RST', 'REQ', 'no', 'ACC', 'CLO',
       'URN', 'PAR'], dtype=object)

In [10]:
import redis
import json

def fetch_data_from_db():
    # Setup Redis connection
    r = redis.Redis(host="5.tcp.eu.ngrok.io", port=15140, db=0)

    # Subscribe to the channel
    pubsub = r.pubsub()
    pubsub.subscribe("processed_data_channel")

    print("Subscribed to 'processed_data_channel'. Waiting for data...")
    # Listen for new messages
    for message in pubsub.listen():
        if message["type"] == "message":
            data = json.loads(message["data"])
            print("Received data:", data)
            df = pd.DataFrame([data])
            return df

input = fetch_data_from_db()

Subscribed to 'processed_data_channel'. Waiting for data...
Received data: {'dur': '0', 'proto': 'TCP', 'service': 'HTTPS', 'state': '', 'stcpb': '808560832', 'dtcpb': '3086082638', 'synack': '0', 'ackdat': '0', 'trans_depth': '0', 'response_body_len': '0', 'ct_src_dport_ltm': '0', 'ct_dst_sport_ltm': '0', 'is_ftp_login': '0', 'ct_ftp_cmd': '0', 'ct_flw_http_mthd': '0', 'is_sm_ips_ports': '0'}


In [13]:
input_df = input.copy()
input_df = input_df.astype(
    {
        "dur": "float64",
        "synack": "float64",
        "ackdat": "float64",
        "stcpb": "int64",
        "dtcpb": "int64",
        "trans_depth": "int64",
        "response_body_len": "int64",
        "ct_src_dport_ltm": "int64",
        "ct_dst_sport_ltm": "int64",
        "is_ftp_login": "int64",
        "ct_ftp_cmd": "int64",
        "ct_flw_http_mthd": "int64",
        "is_sm_ips_ports": "int64",
    }
)

input_df[input_df.select_dtypes(include="object").columns] = input_df[
    input_df.select_dtypes(include="object").columns
].apply(lambda x: x.str.lower())

input_df

,dur,proto,service,state,stcpb,dtcpb,synack,ackdat,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports
0,0.0,tcp,https,,808560832,3086082638,0.0,0.0,0,0,0,0,0,0,0,0


In [16]:
df.shape[0]

129859

In [17]:
df["status"].unique()

KeyError: 'status'

In [24]:
df[df['attack_cat'] == "Exploits"].head(20)

,dur,proto,service,state,stcpb,dtcpb,synack,ackdat,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,attack_cat
2,0.000010,unas,-,INT,0,0,0.000000,0.000000,0,0,5,5,0,0,0,0,Exploits
7,0.400044,tcp,http,FIN,2798288347,3252340382,0.061422,0.056904,1,0,1,1,0,0,1,0,Exploits
14,0.000005,nvp,-,INT,0,0,0.000000,0.000000,0,0,10,10,0,0,0,0,Exploits
15,0.000006,unas,-,INT,0,0,0.000000,0.000000,0,0,1,1,0,0,0,0,Exploits
17,1.770958,tcp,http,FIN,2912657296,727007899,0.103934,0.122443,1,829,1,1,0,0,1,0,Exploits
18,0.630186,tcp,-,FIN,2847175536,1717680105,0.095129,0.038752,0,0,1,1,0,0,0,0,Exploits
22,0.848387,tcp,http,FIN,2381760796,282086,0.042593,0.068765,1,24064,1,1,0,0,1,0,Exploits
26,0.237983,tcp,-,FIN,3230617250,3235058402,0.009707,0.050064,0,0,1,1,0,0,0,0,Exploits
27,0.000009,sctp,-,INT,0,0,0.000000,0.000000,0,0,1,1,0,0,0,0,Exploits
33,2.359291,tcp,smtp,FIN,3193484793,594356036,0.047263,0.088712,0,0,1,1,0,0,0,0,Exploits
